# Forearm Assembly

Use CSYS to create a scene

In [1]:
import bumblebee as bb

In [2]:
assm = bb.CSYS()
assm.ax.set_xlim([-10, 10])
assm.ax.set_ylim([-10, 10])
assm.ax.set_zlim([-10,10])
assm.plot()

Add the forearm to the assembly

In [3]:
forearm = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\Stewart\LowPoly\Arm.stl", name="Forearm", units='cm')
assm.add(forearm)
forearm.rotate_from_euler('z', -90, degrees=True)
forearm.translate([0,-71.5935,0])

Add eblow to the assembly

In [4]:
elbow = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\forearm_elbow.stl", name="ForarmElbow", units='cm')
assm.add(elbow)
elbow.rotate_from_euler('y', -90, degrees=True)
elbow.rotate_from_euler('z', 180, degrees=True)
elbow.translate([2,0,2])

Add frames representing the joint, and end of the link to be used in kinematic calcs

In [5]:
joint = bb.Frame(name='Joint')
assm.add(joint)
joint.translate([0,2.7,-2.375])

M = bb.Frame(name='M_pose')
assm.add(M)
M.translate([0,-71.5935*2,0])

In [6]:
forearm_assm = bb.RigidCollection(name='ForearmAssm')
for body in [forearm, elbow, joint, M]:
    forearm_assm.add(body)
    assm.tree.remove_node(body)

forearm_assm.bind(assm)
assm.tree.add_node(forearm_assm)

In [7]:
eccentric = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\elbow_eccentric.stl", name="Eccentric", units='cm')

In [8]:
assm.add(eccentric)

In [9]:
eccentric.rotate_from_euler('y', 90, degrees=True)

In [10]:
eccentric.translate([-0.5, 0.7, 2+0.375+0.375])

In [11]:
assm2 = bb.CSYS()
assm2.ax.set_xlim([-10, 10])
assm2.ax.set_ylim([-10, 10])
assm2.ax.set_zlim([-10,10])
assm2.plot()

In [12]:
whisker = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\whisker.stl", name="Whisker", units='cm')
assm2.add(whisker)

In [13]:
whisker.rotate_from_euler('xz', [-90,90], degrees=True)

In [14]:
whisk_elbow = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\whisker_elbow.stl", name="WhiskerElbow", units='cm')
assm2.add(whisk_elbow)

In [15]:
whisk_elbow.rotate_from_euler('yz', [-90,180], degrees=True)

In [16]:
whisk_elbow.translate([0,2,0.26])

In [17]:
whisk_joint = bb.Frame(name='WhiskerJoint')
assm2.add(whisk_joint)
whisk_joint.translate([2.5, -1.5, -0.19])

In [18]:
whisk_M = bb.Frame(name='WhiskerM')
assm2.add(whisk_M)
whisk_M.translate([2.5, 46.231+1.5, 0.902-1])

In [19]:
whisk_collision = bb.Frame(name='Collision')
assm2.add(whisk_collision)
whisk_collision.translate([2.5, 46.231/2, 0])

In [20]:
# assm2.ax.set_ylim([40, 60])

In [21]:
whisker_assm = bb.RigidCollection(name='WhiskerAssm')
for body in [whisker, whisk_elbow, whisk_joint, whisk_M, whisk_collision]:
    whisker_assm.add(body)
    assm2.tree.remove_node(body)

whisker_assm.bind(assm2)
assm2.tree.add_node(whisker_assm)

In [22]:
assm.plot()

In [23]:
assm.add(whisker_assm)

In [24]:
whisker_assm.update()

In [25]:
whisker_assm.translate([-2.5, 0.7+1.5, 2+0.375+0.375+0.19])

In [26]:
assm3 = bb.CSYS()
assm3.ax.set_xlim([-10, 10])
assm3.ax.set_ylim([-10, 10])
assm3.ax.set_zlim([-10,10])
assm3.plot()

In [27]:
antenna = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\antenna.stl", name="Antenna", units='cm')
assm3.add(antenna)

In [28]:
antenna_assm = bb.RigidCollection(name='AntennaAssm')
for body in [antenna]:
    antenna_assm.add(body)
    assm3.tree.remove_node(body)

antenna_assm.bind(assm3)
assm3.tree.add_node(antenna_assm)

In [29]:
antenna_joint = bb.Frame(name='AntennaJoint')
antenna_assm.add(antenna_joint)
antenna_joint.translate([-3 + 0.28, 1.464, 0])

In [30]:
antenna_M = bb.Frame(name='AntennaM')
antenna_assm.add(antenna_M)
antenna_M.translate([-3 + 0.28, 45.581+1.464+1.132, 0])

In [31]:
assm.plot()

In [32]:
assm.add(antenna_assm)

In [33]:
antenna_assm.update()

In [34]:
antenna_assm.rotate_from_euler('x', 90, degrees=True)

In [35]:
antenna_assm.translate(whisk_joint.tf[0:3,3] - antenna_joint.tf[0:3,3])

In [36]:
antenna_assm.translate([0,0,-0.375])

In [37]:
import numpy as np
from scipy.spatial.transform import Rotation

whisk_vec = whisk_M.tf[1:3,3] - whisk_joint.tf[1:3,3]
R = Rotation.from_euler('x', 90-42.16, degrees=True) # from the antenna
loc = R.as_matrix() @ np.array([0, whisk_vec[0], whisk_vec[1]])

In [38]:
T=np.eye(4)
T[0:3,0:3] = R.as_matrix()
T[0:3,3] = loc + whisk_joint.tf[0:3, 3]

rel_pose = whisk_M.inv_tf @ whisker_assm.tf

newT = T @ rel_pose
whisker_assm.tf = newT.copy()
whisker_assm.update()

In [39]:
assm.ax.set_xlim([-100, 100])
assm.ax.set_ylim([-100, 100])
assm.ax.set_zlim([-100,100])

(-100.0, 100.0)

Save to file for later use

In [40]:
full_assm = bb.RigidCollection(name='ForearmAsm')
for body in [forearm_assm, whisker_assm, antenna_assm]:
    full_assm.add(body)
full_assm.to_json('Forearm')